# Let's discover Linked Open Data

In [1]:
# pretty display of byte serialization
def pprint(msg):
    msg = msg.decode('utf-8')
    for l in msg.split('\n'):
        if l.strip():
            print(l)

## First Graph with Donna
In this chapter you will learn about building simple graph :

- Talk a bit about identifier and URL
- Give a few though about data representation (triple : subject, predicate, object)
- Display graph


In [2]:
# create your first graph
from rdflib import Graph, URIRef

g = Graph(identifier=URIRef('http://example.org/bob_donna_linda'))
len(g)

0

In [3]:
# Introducing Donna
URIRef('http://example.org/bob_donna_linda/donna')

rdflib.term.URIRef('http://example.org/bob_donna_linda/donna')

http://example.org/people/donna is an *URI*. It uses classical web URL and it's a unique identifier for Donna

In [4]:
# Donna is a person
g.add( (URIRef('http://example.org/bob_donna_linda/donna'), 
        URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), 
        URIRef('http://xmlns.com/foaf/0.1/Person')) )
len(g)

1

In [5]:
# let's display our Graph content
content = g.serialize(format='turtle')
pprint(content)

@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
<http://example.org/bob_donna_linda/donna> a <http://xmlns.com/foaf/0.1/Person> .


## Simplify writing and reading

One a the few very important will for Python is readibility, because you spend more time reading code than writing some. So let's see how to simplify writing and reading with a few tricks.

- Use our own Namespace
- Use most current Namespaces
- Display graph in JSON-LD format

In [6]:
# but it's quite hard to write and hard to read. Let's find out an easier way to do the same
# we will use Namespace
from rdflib import Namespace

# Here is Donna (again)
n = Namespace("http://example.org/bob_donna_linda/")
n.donna

rdflib.term.URIRef('http://example.org/bob_donna_linda/donna')

In [7]:
# And there is pre built namespaces for most current RDF specs
from rdflib.namespace import DC, FOAF, RDF, RDFS
RDF.type

rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')

In [8]:
# Here is bob, it's also a person
g.add( (n.bob, RDF.type, FOAF.Person) )
# bob knows Donna
g.add( (n.bob, FOAF.knows, n.donna) )
len(g)

3

In [9]:
# let's add contexte information to make more readable display
g.bind("", n)
g.bind("foaf", FOAF)

# Display our graph respecting JSON-LD format
from rdflib import plugin
from rdflib.serializer import Serializer

context = dict(g.namespaces())
pprint(g.serialize(format='json-ld', context=context))

{
  "@context": {
    "": "http://example.org/bob_donna_linda/",
    "foaf": "http://xmlns.com/foaf/0.1/",
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "xml": "http://www.w3.org/XML/1998/namespace",
    "xsd": "http://www.w3.org/2001/XMLSchema#"
  },
  "@graph": [
    {
      "@id": "http://example.org/bob_donna_linda/bob",
      "@type": "http://xmlns.com/foaf/0.1/Person",
      "http://xmlns.com/foaf/0.1/knows": {
        "@id": "http://example.org/bob_donna_linda/donna"
      }
    },
    {
      "@id": "http://example.org/bob_donna_linda/donna",
      "@type": "http://xmlns.com/foaf/0.1/Person"
    }
  ],
  "@id": "http://example.org/bob_donna_linda"
}


## Complete our graph

Graph contains lot of links between URI but it can also contains Literal, raw data like names, ages and so on.

- String literal
- Integer Literal
- Language

In [10]:
# complete Donna profile with her identity
from rdflib import Literal

g.add( (n.donna, FOAF.name,      Literal("Donna Fales")) )
g.add( (n.donna, FOAF.firstName, Literal("Donna")) )
g.add( (n.donna, FOAF.lastName,  Literal("Fales")) )

len(g)

6

In [11]:
# complete Bob with his age
from rdflib import XSD
g.add( (n.bob,   FOAF.age,       Literal('42', datatype=XSD.integer)) )
len(g)

7

In [12]:
# complete with Bob's label which vary according to language
g.add( (n.bob,   RDFS.label,     Literal('Bob',    lang='en') ) )
g.add( (n.bob,   RDFS.label,     Literal('Robert', lang='fr') ) )
len(g)

9

In [13]:
pprint(g.serialize(format='turtle'))

@prefix : <http://example.org/bob_donna_linda/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
:bob a foaf:Person ;
    rdfs:label "Bob"@en,
        "Robert"@fr ;
    foaf:age 42 ;
    foaf:knows :donna .
:donna a foaf:Person ;
    foaf:firstName "Donna" ;
    foaf:lastName "Fales" ;
    foaf:name "Donna Fales" .


In [14]:
# what we know about Donna
g.add( (n.donna, FOAF.nick,  Literal("Doudie", lang="en")) )
g.add( (n.donna, FOAF.nick,  Literal("Dudu", lang="es")) )
g.add( (n.donna, FOAF.mbox,  URIRef("mailto:donna@example.org")) )

# what we know about Bob
g.add( (n.bob,   FOAF.name,  Literal('Bob')) )

# introducting Linda !
g.add( (n.linda, RDF.type,   FOAF.Person) )
g.add( (n.linda, FOAF.name,  Literal('Linda') ) )
g.add( (n.bob,   FOAF.knows, n.linda) )

len(g)

16

## Navigating through the graph

Graph are a collection of triple

- Display all Subject, Predicate, Object list
- Interrogate graph with simple methods


In [15]:
# Display all triples
for s, p, o in g:
    print(s, p, o)

http://example.org/bob_donna_linda/donna http://xmlns.com/foaf/0.1/nick Dudu
http://example.org/bob_donna_linda/donna http://xmlns.com/foaf/0.1/name Donna Fales
http://example.org/bob_donna_linda/bob http://xmlns.com/foaf/0.1/age 42
http://example.org/bob_donna_linda/donna http://xmlns.com/foaf/0.1/firstName Donna
http://example.org/bob_donna_linda/donna http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
http://example.org/bob_donna_linda/bob http://xmlns.com/foaf/0.1/knows http://example.org/bob_donna_linda/linda
http://example.org/bob_donna_linda/donna http://xmlns.com/foaf/0.1/mbox mailto:donna@example.org
http://example.org/bob_donna_linda/bob http://www.w3.org/2000/01/rdf-schema#label Bob
http://example.org/bob_donna_linda/bob http://www.w3.org/2000/01/rdf-schema#label Robert
http://example.org/bob_donna_linda/bob http://xmlns.com/foaf/0.1/knows http://example.org/bob_donna_linda/donna
http://example.org/bob_donna_linda/bob http://xmlns.com/foaf/0.1/n

In [20]:
# Find out all Bob's label
masque = (n.bob, RDFS.label, None)
list(g.triples(masque))

[(rdflib.term.URIRef('http://example.org/bob_donna_linda/bob'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('Bob', lang='en')),
 (rdflib.term.URIRef('http://example.org/bob_donna_linda/bob'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
  rdflib.term.Literal('Robert', lang='fr'))]

In [26]:
# let's make graph's bob
bobgraph = Graph(namespace_manager=g.namespace_manager)
bobgraph += g.triples( (n.bob, None, None) )
pprint(bobgraph.serialize(format='turtle'))

@prefix : <http://example.org/bob_donna_linda/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
:bob a foaf:Person ;
    rdfs:label "Bob"@en,
        "Robert"@fr ;
    foaf:age 42 ;
    foaf:knows :donna,
        :linda ;
    foaf:name "Bob" .


In [17]:
# All named triples
list(g.triples( (None, FOAF.name, None) ))

[(rdflib.term.URIRef('http://example.org/bob_donna_linda/donna'),
  rdflib.term.URIRef('http://xmlns.com/foaf/0.1/name'),
  rdflib.term.Literal('Donna Fales')),
 (rdflib.term.URIRef('http://example.org/bob_donna_linda/linda'),
  rdflib.term.URIRef('http://xmlns.com/foaf/0.1/name'),
  rdflib.term.Literal('Linda')),
 (rdflib.term.URIRef('http://example.org/bob_donna_linda/bob'),
  rdflib.term.URIRef('http://xmlns.com/foaf/0.1/name'),
  rdflib.term.Literal('Bob'))]

In [23]:
# all persons
list(g.subjects(predicate=RDF.type, object=FOAF.Person))

[rdflib.term.URIRef('http://example.org/bob_donna_linda/linda'),
 rdflib.term.URIRef('http://example.org/bob_donna_linda/donna'),
 rdflib.term.URIRef('http://example.org/bob_donna_linda/bob')]

In [ ]:
# addition of graph
g2 = Graph()
triple = (n.swann, RDF.type, FOAF.person)
g2.add(triple)
g3 = g + g2
for s, p, o in g3:
    print(s,p,o)

## Sparql querying

- simple Select query
- construct

In [28]:
query = """
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
CONSTRUCT { ?s foaf:name ?o . }
WHERE {
  ?s foaf:name ?o .
}
"""
sparql_result = g.query(query)
pprint(sparql_result.serialize(format='turtle'))

@prefix ns1: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
<http://example.org/bob_donna_linda/bob> ns1:name "Bob" .
<http://example.org/bob_donna_linda/donna> ns1:name "Donna Fales" .
<http://example.org/bob_donna_linda/linda> ns1:name "Linda" .


In [33]:
# utilisation de contexte
# utilisation de binding (ie valeur fixée dans la query)
ns = dict(foaf=FOAF)
query_result = g.query('SELECT ?name WHERE { ?p foaf:name ?name }',
                   initNs=ns,
                   initBindings={'p' : n.bob})
for row in query_result:
    print(row)

(rdflib.term.Literal('Bob'),)


In [35]:
# get a new graph from a query
n3data = """\
@prefix : <http://www.snee.com/ns/demo#> .

:Jane :hasParent :Gene .
:Gene :hasParent :Pat ;
      :gender    :female .
:Joan :hasParent :Pat ;
      :gender    :female .
:Pat  :gender    :male .
:Mike :hasParent :Joan ."""
gc1 = Graph().parse(data=n3data, format="n3")

cq = """\
CONSTRUCT { ?p :hasGrandfather ?g . }

WHERE {?p      :hasParent ?parent .
       ?parent :hasParent ?g .
       ?g      :gender    :male .
}"""

nsdict = {'':"http://www.snee.com/ns/demo#"}
result_graph = gc1.query(cq, initNs=nsdict)

newg = Graph().parse(data=result_graph.serialize(format='xml'))
pprint(newg.serialize(format="n3"))

@prefix ns1: <http://www.snee.com/ns/demo#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
ns1:Jane ns1:hasGrandfather ns1:Pat .
ns1:Mike ns1:hasGrandfather ns1:Pat .


## Conjunctive Graph

In [ ]:
from rdflib import ConjunctiveGraph

gc = ConjunctiveGraph()
# graph nommé numéro 1
c1 = URIRef("http://example.org/mygraph1")
# graph nommé numéro 2
c2 = URIRef("http://example.org/mygraph2")
print("le conjunctive graph est vide : ", len(gc))


bob = URIRef(u'urn:bob')
likes = URIRef(u'urn:likes')
pizza = URIRef(u'urn:pizza')
gc1 = gc.get_context(c1)
gc1.add((bob, likes, pizza))
print("Nb éléments dans gc1: ", len(gc1))
print("Nb éléments dans g: ", len(gc))

tom = URIRef(u'urn:tom')
gc2 = gc.get_context(c2)
gc2.add((tom, likes, pizza))
gc2.add((bob, likes, pizza))
print("Nb éléments dans gc2: ", len(gc2))
print("Nb éléments dans g: ", len(gc))

list(gc.contexts())


In [ ]:
print(g.identifier)
g3 = gc.get_context(g.identifier)
print(g3.identifier)
for s, p, o in g3:
    print(s, p, o)

In [ ]:
# insert quads
# g3.addN( (bob, likes, pizza, g3.identifier) )

for i in (bob, likes, pizza, g3.identifier):
    print(i)

for s, p, o, c in (bob, likes, pizza, g3.identifier):
    print(s, p, o, c)